In [161]:
import random
import numpy as np
import secrets
import sys

In [162]:
constants = {
    "BASEFEE_MAX_CHANGE_DENOMINATOR": 8,
    "TARGET_GAS_USED": 10000000,
    "MAX_GAS_EIP1559": 20000000,
    "EIP1559_DECAY_RANGE": 800000,
    "EIP1559_GAS_INCREMENT_AMOUNT": 10,
    "INITIAL_BASEFEE": 1 * (10 ** 9),
    "PER_TX_GASLIMIT": 8000000,
    "SIMPLE_TRANSACTION_GAS": 21000,
}

In [171]:
# All users want to fit a single 21k gas tx
# If they have to wait 10 blocks and pay 5 Gwei / gas
# the total fee is 105k Gwei
# Value must be >= 105k for payoff to be ever positive
# For our User class, value == value per gwei

rng = np.random.default_rng()

def get_basefee_bounds(basefee, blocks):
    lb = basefee * (1 - 1.0 / constants["BASEFEE_MAX_CHANGE_DENOMINATOR"])
    ub = basefee * (1 + 1.0 / constants["BASEFEE_MAX_CHANGE_DENOMINATOR"])
    return { "lb": lb, "ub": ub }

class Transaction:
    
    def __init__(self, gas_premium, maxfee, gas_used = constants["SIMPLE_TRANSACTION_GAS"]):
        self.gas_premium = gas_premium
        self.maxfee = maxfee
        self.gas_used = gas_used
        self.tx_hash = secrets.token_bytes(8)
        
    def __str__(self):
        return f"Transaction {self.tx_hash.hex()}: maxfee {self.maxfee}, gas_premium {self.gas_premium}"
    
    def is_valid(self, basefee):
        return self.maxfee >= basefee
    
    def gas_price(self, basefee):
        return min(self.maxfee, basefee + self.gas_premium)
    
    def tip(self, basefee):
        return self.gas_price(basefee) - basefee

class Block:
    
    def __init__(self, txs, parent_hash):
        self.txs = txs
        self.block_hash = secrets.token_bytes(8)
        self.parent_hash = parent_hash
        
    def __str__(self):
        return "Block:\n" + "\n".join([tx.__str__() for tx in self.txs])

class Chain:
    
    def __init__(self):
        self.blocks = {}
        self.current_head = (0).to_bytes(8, sys.byteorder)
        
    def add_block(self, block):
        self.blocks[block.block_hash] = block
        self.current_head = block.block_hash
    
class User:
    
    def __init__(self, wakeup_block):
        self.wakeup_block = wakeup_block
        self.value = int(random.random() * 20 * (10 ** 9))
        
    def gas_price(self, basefee):
        tx_parameters = self.decide_parameters()
        return min(tx_parameters["maxfee"], basefee + tx_parameters["gas_premium"])
    
    def transact(self):
        params = self.decide_parameters()
        tx = Transaction(
            maxfee = params["maxfee"],
            gas_premium = params["gas_premium"],
        )
        return tx
    
class AffineUser(User):
    
    def __init__(self, wakeup_block):
        super().__init__(wakeup_block)
        self.cost_per_unit = int(random.random() * (10 ** 9))
    
    def decide_parameters(self):
        return {
            "maxfee": self.value - self.expected_time() * self.cost_per_unit,
            "gas_premium": 2 * (10 ** 9),
        }
    
    def expected_time(self):
        return 5
    
    def worst_expected_basefee(self, current_basefee):
        basefee_bounds = get_basefee_bounds(current_basefee, self.expected_time())
        return basefee_bounds["ub"]
    
    def transact(self, basefee):
        params = self.decide_parameters()
        expected_block = self.wakeup_block + self.expected_time()
        payoff = self.payoff(expected_block, self.worst_expected_basefee(basefee))

        if payoff < 0 or params["maxfee"] < 0:
            return None
        else:
            return super().transact()
    
    def payoff(self, included_block, basefee):
        tx_parameters = self.decide_parameters()
        return self.value - self.cost_per_unit * (included_block - self.wakeup_block) - self.gas_price(basefee)
    
    def __str__(self):
        return f"Affine User with value {self.value} and cost {self.cost_per_unit}"
    
class DiscountUser(User):
        
    def __init__(self, wakeup_block):
        super().__init__(wakeup_block)
        self.discount_rate = 0.01
    
    def decide_parameters(self):
        return {
            "maxfee": self.value * (1 - self.discount_rate) ** self.expected_time(),
            "gas_premium": 2,
        }
    
    def payoff(self, included_block, basefee):
        tx_parameters = self.decide_parameters()
        return self.value * (1 - self.discount_rate) ** (included_block - self.wakeup_block) - self.gas_price(basefee)
    
    def __str__(self):
        return f"Discount User with value {self.value} and discount rate {self.discount_rate}"

class TxPool:
    
    def __init__(self):
        self.txs = {}
        self.pool_length = 0
    
    def add_txs(self, txs):
        for tx in txs:
            self.txs[tx.tx_hash] = tx
        self.pool_length += len(txs)
            
    def remove_txs(self, tx_hashes):
        for tx_hash in tx_hashes:
            del(self.txs[tx_hash])
        self.pool_length -= len(tx_hashes)
            
    def __str__(self):
        return "\n".join([tx.__str__() for tx in self.txs.values()])
    
def spawn_demand(timestep, demand_lambda):
    real = rng.poisson(demand_lambda)
    new_users = [AffineUser(timestep) for i in range(real)]
    return new_users

def decide_transactions(demand, basefee = 1):
    # User side
    txs = []
    for user in demand:
        tx = user.transact(basefee)
        if not tx is None:
            txs.append(tx)
    return txs

def select_transactions(txpool, basefee = 1):
    # Miner side
    max_tx_in_block = int(constants["MAX_GAS_EIP1559"] / constants["SIMPLE_TRANSACTION_GAS"])
    sorted_valid_demand = sorted([tx for tx in txpool.txs.values() if tx.is_valid(basefee)], key = lambda tx: -tx.tip(basefee))
    selected_txs = sorted_valid_demand[0:max_tx_in_block]
    return selected_txs

def update_basefee(block, basefee):
    gas_used = sum([tx.gas_used for tx in block.txs])
    delta = gas_used - constants["TARGET_GAS_USED"]
    return basefee + basefee * delta // constants["TARGET_GAS_USED"] // constants["BASEFEE_MAX_CHANGE_DENOMINATOR"]

In [173]:
txpool = TxPool()
basefee = constants["INITIAL_BASEFEE"]
chain = Chain()

for t in range(100):
    users = spawn_demand(t, 1000)
    decided_txs = decide_transactions(users, basefee)
    txpool.add_txs(decided_txs)
    selected_txs = select_transactions(txpool, basefee)
    block = Block(txs = selected_txs, parent_hash = chain.current_head)
    txpool.remove_txs([tx.tx_hash for tx in selected_txs])
    chain.add_block(block)
    basefee = update_basefee(block, basefee)
    
    print(f"basefee = {basefee / (10 ** 9)}, {len(users)} users, {len(decided_txs)} txs, {len(selected_txs)} included transactions, {txpool.pool_length} pool length")

basefee = 1.08815, 985 users, 863 txs, 812 included transactions, 51 pool length
basefee = 1.182642225, 973 users, 861 txs, 807 included transactions, 105 pool length
basefee = 1.281304152, 969 users, 857 txs, 794 included transactions, 168 pool length
basefee = 1.393242085, 991 users, 861 txs, 809 included transactions, 220 pool length
basefee = 1.491918455, 942 users, 811 txs, 746 included transactions, 285 pool length
basefee = 1.621864552, 1008 users, 880 txs, 808 included transactions, 357 pool length
basefee = 1.753336947, 997 users, 868 txs, 785 included transactions, 440 pool length
basefee = 1.880278541, 978 users, 842 txs, 752 included transactions, 530 pool length
basefee = 2.028750035, 990 users, 868 txs, 777 included transactions, 621 pool length
basefee = 2.200661241, 1023 users, 899 txs, 799 included transactions, 721 pool length
basefee = 2.390605814, 1016 users, 907 txs, 805 included transactions, 823 pool length
basefee = 2.59255224, 1040 users, 916 txs, 798 included 